In [7]:
from ultralytics import YOLO
import cv2
import numpy as np
import torch
from transformers import AutoImageProcessor
from transformers import AutoModelForObjectDetection
from transformers import pipeline
from PIL import Image, ImageDraw

In [20]:
url = "./detr_licence/272.png"
image = Image.open(url).convert('RGB')


obj_detector = pipeline(task="object-detection", model="./detr_new_data")
result = obj_detector(image)

In [4]:
onnx_model = YOLO('./license-plate-recognition/yolo/runs/detect/train/weights/best.onnx')


In [26]:
results = onnx_model(image)


0: 640x640 3 License_Plates, 34.5ms
Speed: 3.1ms preprocess, 34.5ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)


In [30]:
results[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0., 0.])
conf: tensor([0.6423, 0.5550, 0.2734])
data: tensor([[2.3448e+02, 8.8987e+02, 3.5479e+02, 9.4229e+02, 6.4226e-01, 0.0000e+00],
        [2.3397e+03, 3.9747e+02, 2.4085e+03, 4.3057e+02, 5.5503e-01, 0.0000e+00],
        [2.0084e+03, 4.6833e+02, 2.0655e+03, 5.0378e+02, 2.7343e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (1440, 2560)
shape: torch.Size([3, 6])
xywh: tensor([[ 294.6314,  916.0793,  120.3111,   52.4136],
        [2374.1030,  414.0193,   68.7852,   33.0957],
        [2036.9253,  486.0552,   57.1084,   35.4419]])
xywhn: tensor([[0.1151, 0.6362, 0.0470, 0.0364],
        [0.9274, 0.2875, 0.0269, 0.0230],
        [0.7957, 0.3375, 0.0223, 0.0246]])
xyxy: tensor([[ 234.4758,  889.8726,  354.7870,  942.2861],
        [2339.7104,  397.4714, 2408.4956,  430.5671],
        [2008.3711,  468.3342, 2065.4795,  503.7761]])
xyxyn: tensor([[0.0916, 0.6180, 0.1386, 0.6544],
        [0.9139, 0.2760, 0.

In [31]:

yellow = (0, 255, 255) # in BGR 
font = cv2.FONT_HERSHEY_SIMPLEX
stroke = 2

# Path to the video file
video_path = 'TEST.mp4'

# Open the video
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Read and display frames in a loop
while True:
    (ret, frame) = cap.read()  # Read a frame
    if not ret:
        print("Reached end of video or failed to read, exiting...")
        break  # Exit the loop if there are no frames to read

    # convert the image from NumPy array into a PIL image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(img)


    results = onnx_model(image)[0].boxes
    for score, label, box in zip(results.conf, results.cls, results.xyxy):
        box = [round(i, 2) for i in box.tolist()]
        print(
            f"Detected Licence with confidence "
            f"{round(score.item(), 3)} at location {box}"        
        )

        # draw the bounding box
        cv2.rectangle(frame, 
                      (int(box[0]), int(box[1])),   # x1, y1
                      (int(box[2]), int(box[3])),   # x2, y2
                      yellow, 
                      stroke)
        
        # display the label
        cv2.putText(frame, 
                    'license', # label
                    (int(box[0]), int(box[1]-10)),       # x1, y1
                    font, 
                    1, 
                    yellow, 
                    stroke, 
                    cv2.LINE_AA)

    cv2.imshow('Video Playback', frame)  # Display the frame

    # Wait for 25 ms before moving on to the next frame
    # This will play the video at about 40 frames per second
    # Adjust the wait time as per your video's fps (use int(1000/fps))
    if cv2.waitKey(25) & 0xFF == ord('q'):
        print("Video playback interrupted by user.")
        break  # Exit the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF 
    if key == ord("q"):    # Press q to break out of the loop
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


0: 640x640 3 License_Plates, 32.4ms
Speed: 2.0ms preprocess, 32.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Detected Licence with confidence 0.642 at location [234.48, 889.87, 354.79, 942.29]
Detected Licence with confidence 0.555 at location [2339.71, 397.47, 2408.5, 430.57]
Detected Licence with confidence 0.273 at location [2008.37, 468.33, 2065.48, 503.78]

0: 640x640 4 License_Plates, 49.5ms
Speed: 2.7ms preprocess, 49.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Detected Licence with confidence 0.666 at location [239.67, 869.88, 354.31, 919.59]
Detected Licence with confidence 0.521 at location [2343.1, 400.34, 2413.6, 432.6]
Detected Licence with confidence 0.458 at location [1978.11, 472.18, 2065.08, 514.07]
Detected Licence with confidence 0.363 at location [570.26, 341.59, 629.73, 373.1]

0: 640x640 5 License_Plates, 36.5ms
Speed: 2.4ms preprocess, 36.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Detecte

-1